In [1]:
import os
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
#import gdal
import geopandas as gpd
from shapely.geometry import Point, Polygon, mapping
import rasterio
import rasterio.plot
import rasterio.mask # an important package which allows you to only extract points within your shapefile
import rasterstats # gives us raster summaries within a shapefile
from rasterstats import zonal_stats # gives us raster summaries within a shapefile
os.getcwd()

'C:\\Users\\leogo\\OneDrive\\Desktop\\PhD\\Analysis\\scripts'

In [2]:
#os.chdir('../../')
os.chdir('../')
#os.chdir("Analysis")
os.getcwd()

'C:\\Users\\leogo\\OneDrive\\Desktop\\PhD\\Analysis'

In [20]:
#world_grid=gpd.read_file(os.path.join('data', 'shapefiles', 'world_grid_100.0km'))
#geometry=world_grid.values


In [7]:
# subsetting shapefile by continent

resolution="100"
file_name="world_grid_"+resolution+".0km"
continent="Africa"
continent_file="africa_grid_"+resolution+".0km"

world_grid=gpd.read_file(os.path.join('data', 'shapefiles', file_name))
world_shapefile=gpd.read_file(os.path.join('data', 'shapefiles', 'World'))
world_geopanda = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) #continents dataframe
continent_shapefile=world_geopanda.loc[world_geopanda["continent"]==continent,["continent","geometry"]]
continent_shapefile["geometry"]=continent_shapefile.envelope
continent_shapefile.columns=["geometry_continent" if col_name=="geometry" else col_name for col_name in continent_shapefile.columns]
#subset_world_shapefile=gpd.overlay(world_shapefile.set_geometry("geometry"), continent_shapefile.set_geometry("geometry_continent"), how='intersection')
subset_world_shapefile=gpd.sjoin(world_shapefile.set_geometry("geometry"), continent_shapefile.set_geometry("geometry_continent"), how='inner')
world_shapefile=None
continent_shapefile=None
countries=subset_world_shapefile["CNTRY_NAME"].unique()


# Getting rid of some european countries
countries_to_exclude=["Portugal", "Spain", "Gibraltar", "Gaza Strip", "Israel", "Jordan", "West Bank", "Saudi Arabia", "Yemen"]
final_countries=pd.Series(countries).isin(countries_to_exclude)==False
final_countries=countries[final_countries]


#subsetting final dataframe
subset_world_shapefile=subset_world_shapefile.loc[subset_world_shapefile["CNTRY_NAME"].isin(final_countries),:]
grid_overlay=gpd.overlay(subset_world_shapefile,world_grid, how="identity")
subset_world_shapefile=None
world_grid=grid_overlay
grid_overlay=None
world_grid.drop(columns=['OBJECTID', 'CNTRY_NAME', 'index_right', 'continent']).to_file(os.path.join('data', 'shapefiles', continent_file))

world_grid=None

KeyboardInterrupt: 

In [30]:
rasters_to_attach={"education":os.path.join("data", "raster","ClimAfr21_education_index","ClimAfr21_education index.tif"),
                  "technological_capital":os.path.join("data", "raster","ClimAfr08_technological_capital_index","ClimAfr08_technological capital index.tif"),
                  "infrastructure":os.path.join("data", "raster","ClimAfr23_infrastructure_index","ClimAfr23_infrastructure index.tif"),
                  "conflictuality":os.path.join("data", "raster","ClimAfr27_conflictuality_index","ClimAfr27_conflictuality index.tif"),
                  #"soil":os.path.join("data", "raster","HWSD_RASTER","hwsd.bil"),
                  "cattle_density":os.path.join("data", "raster","cattle_density_map","Cattle10km_AD_2010_v2_1.tif"),
                  #"AEZ":os.path.join("data", "raster","ClimAfr_AEZ","ClimAf_1_1981_2050_lpjm_mir5_B1_AEZAI_ann_avgCUR05_10km.tif"),
                  "financial_capital":os.path.join("data", "raster","ClimAfr09_financial_capital_index","ClimAfr09_financial capital index.tif"),
                  "rurality":os.path.join("data", "raster","ClimAfr13_rurality_index","ClimAfr13_rurality index.tif"),
                  "gender_gap":os.path.join("data", "raster","ClimAfr15_gender_gap_index","ClimAfr15_gender gap index.tif"),
                  "household_technological_capital":os.path.join("data", "raster","ClimAfr22_household_technology_index","ClimAfr22_household technology index.tif"),
                  "financial_development":os.path.join("data", "raster","ClimAfr24_financial_development_index","ClimAfr24_financial development index.tif"),
                  "richness":os.path.join("data", "raster","ClimAfr25_richness_index","ClimAfr25_richness index.tif"),
                  "governance":os.path.join("data", "raster","ClimAfr26_governance_index","ClimAfr26_governance index.tif"),
                  #"slope":os.path.join("data", "raster","global_slope_median","plate09.bil"),
                  "population_density":os.path.join("data", "raster","gridded_pop_world_population_density_2015","gpw_v4_population_density_adjusted_to_2015_unwpp_country_totals_rev11_2015_2pt5_min.tif"),
                  "institutional_capital":os.path.join("data", "raster","ClimAfr10_institutional_capital_index","ClimAfr10_institutional capital index.tif"),
                  "AEZ":os.path.join("data", "raster","AEZ_2009","AEZ_FINAL.asc"),
                  "time_to_market_20_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_020k.tif"),
                  "time_to_market_50_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_050k.tif"),
                  "time_to_market_100_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_100k.tif"),
                  "time_to_market_250_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_250k.tif"),
                  "time_to_market_500_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_500k.tif")}
                  #"pests_and_disease":os.path.join("data", "raster","Harvest_Choice_Pests_and_Disease","Pests_and_Disease.tiff")}

raster_keys=[value for value, key in  rasters_to_attach.items()]
raster_labels=[key for value, key in  rasters_to_attach.items()]


In [31]:

AEZ_mapping_categories={101:"Temperate / arid",
                    102:"Temperate / Semi-arid",
                    103:"Temperate / sub-humid",
                    104:"Temperate / humid",
                    211:"Subtropic - warm / arid",
                    212:"Subtropic - warm / semiarid",
                    213:"Subtropic - warm / subhumid",
                    214:"Subtropic - warm / humid",
                    221:"Subtropic - cool / arid",
                    222:"Subtropic - cool / semiarid",
                    223:"Subtropic - cool / subhumid",
                    224:"Subtropic - cool / humid",
                    311:"Tropic - warm / arid",
                    312:"Tropic - warm / semiarid",
                    313:"Tropic - warm / subhumid",
                    314:"Tropic - warm / humid",
                    321:"Tropic - cool / arid",
                    322:"Tropic - cool / semiarid",
                    323:"Tropic - cool / subhumid",
                    324:"Tropic - cool / humid",
                    400:"Boreal"}


#filepath=AEZ_raster
#shapefile=world_grid
#data_type="AEZ"
#with rasterio.open(filepath) as src:
    # can check the profile and type of src, but effectively is an open file command
#    affine = src.transform # transforms the raster dataset into the coordinate reference system
#    print("broke before read")
#    #raster_array=rasterio.open(filepath,driver="AAIGrid").read(1)
#    raster_array=src.read()
#    print("broke after open")
#    #raster_array_AEZ=zonal_stats(world_grid, AEZ_raster,categorical=True,category_map=mapping_categories, stats=["majority"])
#    raster_array_AEZ = pd.DataFrame(zonal_stats(shapefile,raster_array, categorical=True,category_map=mapping_categories,affine=affine,stats=["majority"]))
#    print("broke when converting to df")
#raster_statistics_for_grid=pd.concat([shapefile,raster_statistics_for_grid],axis=1)
#grid_raster_merge=raster_statistics_for_grid


In [32]:
#https://gis.stackexchange.com/questions/297076/how-to-calculate-mean-value-of-a-raster-for-each-polygon-in-a-shapefile
def merge_shape_file_with_raster(filepath, shapefile,raster_keys):
    with rasterio.open(filepath) as src:
        # can check the profile and type of src, but effectively is an open file command
        affine = src.transform # transforms the raster dataset into the coordinate reference system
        print("reading "+raster_keys)
        raster_array=rasterio.open(filepath).read(1) # why is this "1" here?
        print("converting to numeric "+raster_keys)

        if raster_keys!="AEZ":
            raster_array=raster_array.astype(float)
        #raster_array[raster_array<-3.4e+38]=np.nan
            raster_array[raster_array<-2e+9]=np.nan
        if raster_keys=="AEZ":
            raster_array=raster_array.astype(int)
        
        print("Zonal Stats "+raster_keys)
        
        if raster_keys!="AEZ":
            raster_statistics_for_grid = pd.DataFrame(zonal_stats(shapefile,raster_array ,affine=affine,stats=[ 'mean', 'median', 'std'])) # calculates raster statistics within boundary of shapefile
       
        if raster_keys=="AEZ":
            raster_statistics_for_grid = pd.DataFrame(zonal_stats(shapefile,raster_array ,affine=affine,stats=["majority"])) # calculates raster statistics within boundary of shapefile

    
    #raster_array=None
    print("Concat "+raster_keys)
    raster_statistics_for_grid=pd.concat([shapefile,raster_statistics_for_grid],axis=1)
    grid_raster_merge=raster_statistics_for_grid
    raster_statistics_for_grid

    print("Changing col names "+raster_keys)
    if raster_keys!="AEZ":
        grid_raster_merge.columns=[column_name.replace("mean","mean_"+raster_keys) if column_name=="mean" else column_name  for column_name in grid_raster_merge.columns]
        grid_raster_merge.columns=[column_name.replace("median","median_"+raster_keys) if column_name=="median" else column_name for column_name in grid_raster_merge.columns]
        grid_raster_merge.columns=[column_name.replace("std","std_"+raster_keys) if column_name=="std" else column_name for column_name in grid_raster_merge.columns]
    if raster_keys=="AEZ":
        grid_raster_merge.columns=[column_name.replace("majority","majority_"+datatype) if column_name=="majority" else column_name for column_name in grid_raster_merge.columns]

    return grid_raster_merge;

In [ ]:
resolution="100"
file_name="world_grid_"+resolution+".0km"
continent="Africa"
continent_file="africa_grid_"+resolution+".0km"

world_grid_old=gpd.read_file(os.path.join('data', 'shapefiles', continent_file))
#world_grid=None
for loop_index in range(len(rasters_to_attach)):
    print("----------------------"+raster_keys[loop_index]+"----------------------------")
    world_grid_new=merge_shape_file_with_raster(filepath=raster_labels[loop_index],  shapefile=world_grid_old, raster_keys=raster_keys[loop_index])
    columns_to_add=["FID"]
    columns_to_add.extend(list(world_grid_new.columns.difference(world_grid_old.columns)))
    print("Merging old and new "+raster_keys[loop_index])
    world_grid_old=pd.merge(world_grid_old,world_grid_new[world_grid_new.columns.intersection(columns_to_add)])  
    #world_grid_new=None
processed_file_name="africa_grid_"+resolution+"processed.csv"
world_grid_old.to_csv(os.path.join("data","processed",processed_file_name))

In [3]:
resolution="100"
file_name="world_grid_"+resolution+".0km"
continent="Africa"
continent_file="africa_grid_"+resolution+".0km"

world_grid_old=gpd.read_file(os.path.join('data', 'shapefiles', continent_file))

In [4]:
world_grid_old.columns

Index(['OBJECTID', 'CNTRY_NAME', 'index_righ', 'continent', 'FID', 'geometry'], dtype='object')

In [ ]:
subsetting_non_na_values=world_grid_old.loc[:,"majority"+"_AEZ"].notna()
grid_raster_merge_non_na=world_grid_old.loc[subsetting_non_na_values,:]

In [ ]:
grid_raster_merge_non_na.plot()